In [1]:
# --- 1. Setup: Imports and Configuration ---

import os
import sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import timm
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
from torchvision.ops import nms

# --- Configuration ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Classifier Config
CLF_MODEL_NAME = 'tf_efficientnet_b5_ns'
CLF_IMG_SIZE = 512
CLF_BATCH_SIZE = 4 # Reduced from 8 to prevent OOM
CLF_MODEL_PATHS = sorted(glob.glob('classifier_fold*_best.pth'))
CLASSES = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
CLASS_MAP_LOWER = {
    'Negative for Pneumonia': 'negative',
    'Typical Appearance': 'typical',
    'Indeterminate Appearance': 'indeterminate',
    'Atypical Appearance': 'atypical'
}

# Detector Config
DET_IMG_SIZE = 640
DET_BATCH_SIZE = 16
DET_MODEL_PATHS = sorted(glob.glob('yolov5_runs/train_cv/yolov5s_fold*/weights/best.pt'))

# Post-Processing & Ensemble Config
# NEW TUNED PARAMS from notebook 07:
TUNED_CONF_THRESHOLD = 0.10
TUNED_NEG_FILTER_THRESHOLD = 0.70
DETECTOR_RAW_CONF_THRESHOLD = 0.001 # Keep this low to get all boxes for subsequent filtering
NMS_IOU_THRESHOLD = 0.5 # This was not tuned, keep as is

# Data Paths
TEST_IMAGE_DIR_3CH = 'test_png_3ch/' # For classifier
TEST_IMAGE_DIR_1CH = 'test_png/'     # For detector
SAMPLE_SUB_PATH = 'sample_submission.csv'

print(f"Using device: {DEVICE}")
print(f"Found {len(CLF_MODEL_PATHS)} classifier models.")
print(f"Found {len(DET_MODEL_PATHS)} detector models.")
print(f"Using TUNED parameters: conf_th={TUNED_CONF_THRESHOLD}, neg_filter_th={TUNED_NEG_FILTER_THRESHOLD}")

Using device: cuda
Found 5 classifier models.
Found 5 detector models.
Using TUNED parameters: conf_th=0.1, neg_filter_th=0.7


In [2]:
# --- 2. Classifier Inference (5-Fold Ensemble) ---

def get_transforms(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

class SIIMCOVIDTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, os.path.basename(image_path).replace('.png', '')

print("--- Starting Classifier Inference ---")

# Load test file list
df_test_imgs = pd.DataFrame({'id': os.listdir(TEST_IMAGE_DIR_3CH)})
df_test_imgs['image_id'] = df_test_imgs['id'].str.replace('.png', '')
df_test_imgs['path'] = TEST_IMAGE_DIR_3CH + df_test_imgs['id']

# Create dataset and dataloader
# FIX: Set num_workers=0 to prevent hanging issue
test_dataset = SIIMCOVIDTestDataset(df_test_imgs['path'].values, transform=get_transforms(CLF_IMG_SIZE))
test_loader = DataLoader(test_dataset, batch_size=CLF_BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# Load all 5 classifier models
models = []
for path in CLF_MODEL_PATHS:
    model = timm.create_model(CLF_MODEL_NAME, pretrained=False, num_classes=len(CLASSES))
    model.load_state_dict(torch.load(path))
    model.to(DEVICE)
    model.eval()
    models.append(model)
print(f"Loaded {len(models)} classifier models onto {DEVICE}")

# Run inference
all_preds = []
with torch.no_grad():
    for images, image_ids in tqdm(test_loader, desc="Classifier Inference"):
        images = images.to(DEVICE)
        
        # Get predictions from all models and average them
        batch_preds = torch.zeros((images.size(0), len(CLASSES)), device=DEVICE)
        for model in models:
            batch_preds += torch.softmax(model(images), dim=1)
        batch_preds /= len(models)
        
        for i, image_id in enumerate(image_ids):
            preds = {f'pred_{cls}': batch_preds[i, j].item() for j, cls in enumerate(CLASSES)}
            preds['image_id'] = image_id
            all_preds.append(preds)

df_image_preds = pd.DataFrame(all_preds)

# Aggregate to study level
test_dcm_paths = glob.glob('test/*/*/*.dcm')
test_map_data = []
for path in test_dcm_paths:
    parts = path.split('/')
    study_id = parts[1]
    image_id = parts[-1].replace('.dcm', '')
    test_map_data.append({'image_id': image_id, 'StudyInstanceUID': study_id})
study_id_map = pd.DataFrame(test_map_data).drop_duplicates()

df_image_preds_with_study = df_image_preds.merge(study_id_map, on='image_id', how='left')

pred_cols = [f'pred_{c}' for c in CLASSES]
df_study_preds = df_image_preds_with_study.groupby('StudyInstanceUID')[pred_cols].mean().reset_index()

print("Classifier inference complete. Study-level predictions created.")

--- Starting Classifier Inference ---


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


Loaded 5 classifier models onto cuda


Classifier Inference:   0%|          | 0/160 [00:00<?, ?it/s]

Classifier Inference:   0%|          | 0/160 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 0 has a total capacity of 23.72 GiB of which 263.12 MiB is free. Process 22789 has 227.00 MiB memory in use. Process 43368 has 599.00 MiB memory in use. Process 54134 has 3.35 GiB memory in use. Process 342427 has 1.80 GiB memory in use. Process 362585 has 11.13 GiB memory in use. Process 369303 has 933.00 MiB memory in use. Process 408019 has 1.75 GiB memory in use. Process 413696 has 1.39 GiB memory in use. Of the allocated memory 1.12 GiB is allocated by PyTorch, and 10.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
# --- 3. Detector Inference (5-Fold Ensemble) ---

print("--- Starting Detector Inference ---")

# Load all 5 detector models
det_models = []
for path in tqdm(DET_MODEL_PATHS, desc="Loading detector models"):
    model = torch.hub.load(
        'ultralytics/yolov5',
        'custom',
        path=path,
        force_reload=True,
        _verbose=False
    )
    model.to(DEVICE).eval()
    model.conf = DETECTOR_RAW_CONF_THRESHOLD # Use a very low threshold to get all possible boxes
    model.iou = NMS_IOU_THRESHOLD
    det_models.append(model)
print(f"Loaded {len(det_models)} detector models onto {DEVICE}")

# Get test image paths
test_image_paths = sorted(glob.glob(f'{TEST_IMAGE_DIR_1CH}/*.png'))
test_image_ids = [os.path.basename(p).replace('.png', '') for p in test_image_paths]
print(f"Found {len(test_image_paths)} test images for detection.")

# Run inference and collect all raw predictions from all models
all_det_preds = []
with torch.no_grad():
    for i in tqdm(range(0, len(test_image_paths), DET_BATCH_SIZE), desc="Detector Inference"):
        batch_paths = test_image_paths[i:i+DET_BATCH_SIZE]
        batch_image_ids = test_image_ids[i:i+DET_BATCH_SIZE]

        batch_model_preds = [[] for _ in range(len(batch_paths))]

        for model in det_models:
            results = model(batch_paths, size=DET_IMG_SIZE)
            preds_df_list = results.pandas().xyxy
            
            for j, preds_df in enumerate(preds_df_list):
                if not preds_df.empty:
                    batch_model_preds[j].append(preds_df[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].values)

        for j, image_id in enumerate(batch_image_ids):
            if batch_model_preds[j]:
                combined_preds = np.vstack(batch_model_preds[j])
                for pred in combined_preds:
                    all_det_preds.append({
                        'image_id': image_id,
                        'x_min': pred[0],
                        'y_min': pred[1],
                        'x_max': pred[2],
                        'y_max': pred[3],
                        'confidence': pred[4]
                    })

df_det_preds_raw = pd.DataFrame(all_det_preds)
print("Detector inference complete. Raw box predictions created.")

--- Starting Detector Inference ---


Loading detector models:   0%|          | 0/5 [00:00<?, ?it/s]

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /app/.cache/torch/hub/master.zip


Loading detector models:   0%|          | 0/5 [00:01<?, ?it/s]

Exception: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.72 GiB of which 251.12 MiB is free. Process 22789 has 227.00 MiB memory in use. Process 43368 has 599.00 MiB memory in use. Process 54134 has 3.35 GiB memory in use. Process 342427 has 1.80 GiB memory in use. Process 362585 has 11.13 GiB memory in use. Process 369303 has 933.00 MiB memory in use. Process 408019 has 1.75 GiB memory in use. Process 413696 has 1.40 GiB memory in use. Of the allocated memory 1.14 GiB is allocated by PyTorch, and 56.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables). Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

In [4]:
# --- 4. Combine, Post-Process, and Format for Submission ---

print("--- Combining, Post-Processing, and Formatting with Tuned Parameters ---")

# --- Step 4.1: Apply Tuned Filtering and NMS ---

# First, filter raw detector predictions by the tuned confidence threshold
print(f"Raw detector predictions from all models: {len(df_det_preds_raw) if 'df_det_preds_raw' in locals() else 'Not generated yet'}")
df_det_conf_filtered = df_det_preds_raw[df_det_preds_raw['confidence'] > TUNED_CONF_THRESHOLD].copy()
print(f"Boxes after confidence filtering (conf > {TUNED_CONF_THRESHOLD}): {len(df_det_conf_filtered)}")

# Merge with study info to get StudyInstanceUID and classifier predictions
df_det_conf_filtered_with_study = df_det_conf_filtered.merge(study_id_map, on='image_id', how='left')
df_det_merged = df_det_conf_filtered_with_study.merge(df_study_preds, on='StudyInstanceUID', how='left')

# Apply classifier-guided filtering using the tuned threshold
print(f"Boxes before classifier filtering: {len(df_det_merged)}")
df_det_filtered = df_det_merged[df_det_merged['pred_Negative for Pneumonia'] < TUNED_NEG_FILTER_THRESHOLD]
print(f"Boxes after classifier filtering (neg_pred < {TUNED_NEG_FILTER_THRESHOLD}): {len(df_det_filtered)}")

final_image_preds = []
# Group by image to apply NMS
for image_id, group in tqdm(df_det_filtered.groupby('image_id'), desc="Applying NMS"):
    boxes = torch.tensor(group[['x_min', 'y_min', 'x_max', 'y_max']].values, dtype=torch.float32)
    scores = torch.tensor(group['confidence'].values, dtype=torch.float32)
    
    # Apply NMS
    keep_indices = nms(boxes, scores, NMS_IOU_THRESHOLD)
    
    final_boxes = boxes[keep_indices].cpu().numpy()
    final_scores = scores[keep_indices].cpu().numpy()
    
    # Format for submission string
    pred_string_parts = []
    for box, score in zip(final_boxes, final_scores):
        pred_string_parts.append(f"opacity {score:.4f} {box[0]:.1f} {box[1]:.1f} {box[2]:.1f} {box[3]:.1f}")
    
    if pred_string_parts:
        final_image_preds.append({
            'id': f"{image_id}_image",
            'PredictionString': " ".join(pred_string_parts)
        })

df_image_sub = pd.DataFrame(final_image_preds)

# --- Step 4.2: Format Study-Level Predictions ---
study_sub_list = []
for _, row in df_study_preds.iterrows():
    study_id = row['StudyInstanceUID']
    pred_strings = []
    for cls in CLASSES:
        pred_strings.append(f"{CLASS_MAP_LOWER[cls]} {row[f'pred_{cls}']:.4f} 0 0 1 1")
    
    study_sub_list.append({
        'id': f"{study_id}_study",
        'PredictionString': " ".join(pred_strings)
    })

df_study_sub = pd.DataFrame(study_sub_list)

# --- Step 4.3: Combine and Create Final Submission File ---
df_submission = pd.concat([df_study_sub, df_image_sub], ignore_index=True)

# Add entries for images with no predicted boxes (submit 'none 1 0 0 1 1')
all_test_image_ids = {f"{img_id}_image" for img_id in test_image_ids}
predicted_image_ids = set(df_image_sub['id'].unique())
missing_image_ids = all_test_image_ids - predicted_image_ids

missing_df = pd.DataFrame([{'id': img_id, 'PredictionString': 'none 1 0 0 1 1'} for img_id in missing_image_ids])
df_submission = pd.concat([df_submission, missing_df], ignore_index=True)

# Reorder to match sample submission
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
df_submission = df_submission.set_index('id').reindex(sample_sub['id']).reset_index()

# Save to file
df_submission.to_csv('submission.csv', index=False)

print("\nFinal submission.csv created successfully with tuned parameters!")
display(df_submission.head())
display(df_submission.tail())
print(f"Total rows in submission: {len(df_submission)}")

--- Combining, Post-Processing, and Formatting with Tuned Parameters ---
Raw detector predictions from all models: Not generated yet


NameError: name 'df_det_preds_raw' is not defined